<img src = "https://www.nzherald.co.nz/resizer/Eo-yZf0zTpughNBjFD4lxh6ZX-c=/620x349/smart/filters:quality(70)/arc-anglerfish-syd-prod-nzme.s3.amazonaws.com/public/PEO6CO5POJHMHOEOPOU5XBA234.jpg" width = 400>
<h1 align=center><font size = 6>A better life across the ditch</font></h1>

### 1. A description of the problem and a discussion of the background. 

For decades, a steady flow of New Zealanders headed to Australia for better employment and lifestyle options. This trend last peaked in 2012 when the Australian economy was booming. Since then the numbers have declined, flat-lining (below 2000 long-term departures) since 2014. However since the start of 2018 the traditional trend has started to re-emerge. Annual net migration for the year to July 2018 was 63,800, according to new numbers released by Stats NZ. That is now down 8,600 from its peak of 74,200 in July 2017 – but still high by historic levels.

There are number of reasons why New Zealanders choose to migrate to Australia but the most significant ones are housing cost, wage, and job opportunities. But for those who failed to conduct research and gather information on life in Australia prior to moving often find that the life style in Australia is vastly different from the life style in New Zealand and this often leads to unsatisfying experience especially given the fact that for majority of migrating population the reason for migration is for financial benefit but not necessarily for change in their living environment. 


### 2. A description of the data and how it will be used to solve the problem.

As such, I will try to address the problem stated above by using location data collected through the Foursquare API. Foursquare allows us to gather a list of recommended venues within a user defined radius of a specified location (e.g. Auckland, Parnell). These data will then be transformed via one hot encoding to be fed into an unsupervised machine learning algorithm called k means clustering where the entire dataset will be clustered into number of clusters according to the similarity between samples (cities in this case) based on its features(frequency of recommended venues) within a cluster. Although this may not be a perfect solution to the problem and there are many aspects to be considered in determining the similarity between cities (e.g. population structure, infrastructure, etc) this is a good starting place and gives us an overall picture of the situation.

In [1]:
from bs4 import BeautifulSoup

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import seaborn as sns

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## Auckland information

Get latitude and longitude information for cities in Auckland from the following site: https://www.latlong.net/category/districts-157-16.html

Remove Westcoast and Hastings from the list as no information on those cities are available on Foursquare.

In [2]:
source = requests.get('https://www.latlong.net/category/districts-157-16.html').text
soup = BeautifulSoup(source, 'lxml')
table = soup.find('table')

my_table = []
for tr in table.find_all('tr'):
    for td in tr.find_all('td'):
        my_table.append(td.text)
        
my_table = np.reshape(my_table,(-1,3))  
df_nz = pd.DataFrame(my_table,columns=['name','Latitude','Longitude'])
df_nz.drop([1,4], inplace=True)
df_nz.reset_index(drop=True)
df_nz['Latitude'] = df_nz['Latitude'].apply(float)
df_nz['Longitude'] = df_nz['Longitude'].apply(float)
df_nz['Borough'] = df_nz['name'].apply(lambda x: x.split(',')[1])
df_nz['Neighbourhood'] = df_nz['name'].apply(lambda x: x.split(',')[0])
df_nz.drop(columns='name', inplace=True)
df_nz.reset_index(inplace=True,drop=True)
df_nz = df_nz[['Borough','Neighbourhood','Latitude','Longitude']]
df_nz.head()

,Borough,Neighbourhood,Latitude,Longitude
0,Auckland,Rosedale,-36.746212,174.737122
1,Auckland,Henderson,-36.873928,174.627731
2,Auckland,Parnell,-36.854065,174.779877


## Cities in Australia

In [3]:
def Borough_return(x):
    x = x.split(',')
    if len(x) == 1:
        return x[0]
    elif len(x) == 2:
        return x[0]
    else:
        return x[1]

def Neigh_return(x):
    x = x.split(',')
    if len(x) == 1:
        return x[0]
    elif len(x) == 2:
        return x[0]
    else:
        return x[0]

Get latitude and longitude information for major cities in Australia.

In [4]:
source_auz = requests.get('https://www.latlong.net/category/cities-14-15.html').text
soup_auz = BeautifulSoup(source_auz, 'lxml')
table_auz = soup_auz.find('table')

my_table_auz = []
for tr in table_auz.find_all('tr'):
    for td in tr.find_all('td'):
        my_table_auz.append(td.text)

my_table_auz = np.reshape(my_table_auz,(-1,3))  
df_auz = pd.DataFrame(my_table_auz,columns=['name','Latitude','Longitude'])
df_auz['Latitude'] = df_auz['Latitude'].apply(float)
df_auz['Longitude'] = df_auz['Longitude'].apply(float)
df_auz['Borough'] = df_auz['name'].apply(Borough_return)
df_auz['Neighbourhood'] = df_auz['name'].apply(Neigh_return)
df_auz.drop(columns='name', inplace=True)
df_auz.reset_index(inplace=True,drop=True)
df_auz = df_auz[['Borough','Neighbourhood','Latitude','Longitude']]
df_auz.head()

,Borough,Neighbourhood,Latitude,Longitude
0,Queensland,Sunshine Coast,-26.650000,153.066666
1,Queensland,Gold Coast,-28.016666,153.399994
2,VIC,Melbourne,-37.840935,144.946457
3,SA,Adelaide,-34.921230,138.599503
4,TAS,Launceston,-41.429825,147.157135


Merge two dataframes.

In [15]:
frames = [df_auz, df_nz]
df_all = pd.concat(frames)
df_all.reset_index(inplace=True,drop=True)
df_all.head()

,Borough,Neighbourhood,Latitude,Longitude
0,Queensland,Sunshine Coast,-26.650000,153.066666
1,Queensland,Gold Coast,-28.016666,153.399994
2,VIC,Melbourne,-37.840935,144.946457
3,SA,Adelaide,-34.921230,138.599503
4,TAS,Launceston,-41.429825,147.157135


## Foursquare API

Let us find trending venues around Parnell, Auckland.

In [6]:
address = 'Auckland, NZ'

geolocator = Nominatim(user_agent="my_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Auckland are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Auckland are -36.8534665, 174.7655514.


In [7]:
CLIENT_ID = 'VFMXFZOMHTXF5DATLEKFWT0MTCMT5AIYZDVBUGUURETN0HKH' # your Foursquare ID
CLIENT_SECRET = 'VIFZAB13AE2AXAUENWQTY1SIFTSGZBRR2HONRCIDQAVVNYU3' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [16]:
neighborhood_latitude = df_all.loc[29, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_all.loc[29, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_all.loc[29, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Parnell are -36.854065, 174.779877.


In [17]:
LIMIT = 100
radius = 1000 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

In [18]:
results = requests.get(url).json()

In [11]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [19]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(10)

,name,categories,lat,lng
0,Non Solo Pizza,Pizza Place,-36.856155,174.780515
1,Woodpecker Hill,Restaurant,-36.854612,174.779968
2,Renkon,Japanese Restaurant,-36.855078,174.780148
3,La Porchetta Parnell,Italian Restaurant,-36.854385,174.779805
4,Espresso Workshop,Coffee Shop,-36.854350,174.779104
5,Gerome,Greek Restaurant,-36.856221,174.780775
6,46 &York,Cocktail Bar,-36.851767,174.778056
7,Chocolate Boutique,Dessert Shop,-36.857209,174.781401
8,Gion,Japanese Restaurant,-36.854071,174.779612
9,La Cigale,Farmers Market,-36.853602,174.783909


In [20]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

87 venues were returned by Foursquare.


Great. Now let's repeat this procedure for all cities in our merged table.

In [22]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [23]:
all_venues = getNearbyVenues(names=df_all['Neighbourhood'],
                                   latitudes=df_all['Latitude'],
                                   longitudes=df_all['Longitude']
                                  )

Sunshine Coast
Gold Coast
Melbourne
Adelaide
Launceston
North Adelaide
Townsville City
Cairns City
Perth
Mildura
Ziyou Today
Coffs Harbour NSW 2450
Orange
Bendigo
Albury
Wollongong
Darwin
Terrey Hills
Bankstown NSW
Westmead
Brisbane
Gladstone QLD
Phillip ACT
Gosford
Mount Gambier
Sydney
Glenore Grove
Rosedale
Henderson
Parnell


In [24]:
all_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adelaide,100,100,100,100,100,100
Albury,46,46,46,46,46,46
Bankstown NSW,55,55,55,55,55,55
Bendigo,54,54,54,54,54,54
Brisbane,100,100,100,100,100,100
Cairns City,14,14,14,14,14,14
Coffs Harbour NSW 2450,12,12,12,12,12,12
Darwin,70,70,70,70,70,70
Gladstone QLD,6,6,6,6,6,6


It looks like the contribution made by each city to the dataset is not balanced which makes sense since some cities are smaller then the others and the population structure also differs for each city. 

In [25]:
print('There are {} uniques categories.'.format(len(all_venues['Venue Category'].unique())))

There are 184 uniques categories.


Now we carry out one hot encoding on the entire dataset so that it can be fed to the k means clustering algorithm.

In [26]:
all_onehot = pd.get_dummies(all_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
all_onehot['Neighborhood'] = all_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [all_onehot.columns[-1]] + list(all_onehot.columns[:-1])
all_onehot = all_onehot[fixed_columns]

all_onehot.head()

,Neighborhood,American Restaurant,Aquarium,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Basketball Court,Beach,Beer Bar,Beer Garden,Big Box Store,Bistro,Bookstore,Botanical Garden,Boutique,Bowling Green,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Café,Candy Store,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Cricket Ground,Cupcake Shop,Cycle Studio,Deli / Bodega,Department Store,Dessert Shop,Discount Store,Dive Bar,Dog Run,Donut Shop,Dumpling Restaurant,Electronics Store,Event Space,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fishing Store,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,German Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,History Museum,Home Service,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Latin American Restaurant,Library,Liquor Store,Lounge,Malay Restaurant,Market,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motel,Movie Theater,Multiplex,Museum,Music Store,Music Venue,National Park,Noodle House,Outdoor Sculpture,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Pharmacy,Pier,Pizza Place,Platform,Playground,Plaza,Pool,Portuguese Restaurant,Post Office,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Road,Sake Bar,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shopping Mall,Shopping Plaza,Skating Rink,Snack Place,South Indian Restaurant,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tennis Stadium,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Water Park,Whisky Bar,Wine Bar,Wine Shop,Zoo
0,Sunshine Coast,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Sunshine Coast,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Sunshine Coast,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Sunshine Coast,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [27]:
all_grouped = all_onehot.groupby('Neighborhood').mean().reset_index()
all_grouped.head()

,Neighborhood,American Restaurant,Aquarium,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Basketball Court,Beach,Beer Bar,Beer Garden,Big Box Store,Bistro,Bookstore,Botanical Garden,Boutique,Bowling Green,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Café,Candy Store,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Cricket Ground,Cupcake Shop,Cycle Studio,Deli / Bodega,Department Store,Dessert Shop,Discount Store,Dive Bar,Dog Run,Donut Shop,Dumpling Restaurant,Electronics Store,Event Space,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fishing Store,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,German Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,History Museum,Home Service,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Latin American Restaurant,Library,Liquor Store,Lounge,Malay Restaurant,Market,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motel,Movie Theater,Multiplex,Museum,Music Store,Music Venue,National Park,Noodle House,Outdoor Sculpture,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Pharmacy,Pier,Pizza Place,Platform,Playground,Plaza,Pool,Portuguese Restaurant,Post Office,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Road,Sake Bar,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shopping Mall,Shopping Plaza,Skating Rink,Snack Place,South Indian Restaurant,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tennis Stadium,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Water Park,Whisky Bar,Wine Bar,Wine Shop,Zoo
0,Adelaide,0.01,0.0,0.01,0.010000,0.0,0.0,0.010000,0.0,0.030000,0.00,0.0,0.020000,0.060000,0.0,0.00,0.00,0.01,0.000000,0.0,0.01,0.00,0.00,0.0,0.00,0.0,0.01,0.0,0.000000,0.000000,0.0,0.010000,0.01,0.000000,0.0,0.050000,0.01,0.000000,0.02,0.0,0.00,0.02,0.040000,0.00,0.000000,0.010000,0.00,0.0,0.02,0.01,0.0,0.0,0.000000,0.01,0.00,0.00,0.0,0.000000,0.00,0.010000,0.00,0.010000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.02,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.01,0.0,0.0,0.01,0.0,0.0,0.0,0.070000,0.0,0.00,0.01,0.0,0.01,0.030000,0.030000,0.010000,0.01,0.00,0.0,0.000000,0.01,0.000000,0.0,0.02,0.020000,0.000000,0.00,0.0,0.010000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.0,0.0,0.02,0.00,0.000000,0.010000,0.01,0.01,0.000000,0.0,0.020000,0.0,0.0,0.00,0.0,0.000000,0.000000,0.020000,0.00,0.00,0.0,0.000000,0.000000,0.0,0.01,0.01,0.000000,0.0,0.03,0.0,0.020000,0.000000,0.0,0.00,0.0,0.01,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.010000,0.01,0.000000,0.01,0.02,0.0,0.010000,0.010000,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.02,0.0,0.010000,0.0,0.00,0.030000,0.01,0.0
1,Albury,0.00,0.0,0.00,0.000000,0.0,0.0,0.021739,0.0,0.000000,0.00,0.0,0.000000,0.065217,0.0,0.00,0.00,0.00,0.021739,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.0,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.0,0.130435,0.00,0.021739,0.00,0.0,0.00,0.00,0.021739,0.00,0.021739,0.000000,0.00,0.

In [ ]:
This concludes the Data section of the project.